Week 6 : Real-World Applications: TF-IDF

In [1]:
%%writefile mapper.py


import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

# stop words
swf = open('/datasets/stop_words_en.txt', 'r')
sws = swf.read().split('\n')
swf.close()


# the word we search
searched_word = 'labor'
# the article for which we compute TF IDF
lookedup_article = 12

docs_with_word = 0

# let's read articles... One article per line, starting with id
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        article_id = int(article_id)
    except ValueError as e:
        print e
        continue

    # split words
    words_with_sws = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    # to lower case
    words_with_sws = [x.lower() for x in words_with_sws]
    # without stop words
    words = [x for x in words_with_sws if x not in sws]
    searched_word_count = words.count(searched_word)
    word_count = len(words)

    # we look for the tf idf for article 12
    # so, lets ouput these values only for this article...
    if article_id == lookedup_article:
        print "%s\t%s\t%s" % (article_id, searched_word_count, word_count)
    # we found one document with the searched word
    # lets use the article id 0 to map that
    if searched_word_count != 0:
        print "0\t1"

Overwriting mapper.py


We leverage the fact that values are sorted by key.
So, we shall compute Dt (docs_with_word) as the reduction of fake article id 0, whose values will be processed first
Then, we shall ouput the tf idf for all articles. Here, only one

In [2]:
%%writefile reducer.py

import sys
from math import log 

swf = open('stop_words_en.txt', 'r')
sws = swf.read().split('\n')
swf.close()

current_key = 0
docs_with_word = 0
idf = None
tf_idf_cur_key = None
searched_word_count = None
word_count = None
for line in sys.stdin:
    try:
        splitted_line = line.strip().split('\t')
        if len(splitted_line) == 2:
            key, dummy = splitted_line
        else:
            key, searched_word_count,word_count = splitted_line
        key = int(key)
        if searched_word_count != None:
            searched_word_count = float(searched_word_count)
        if word_count != None:
            word_count = float(word_count)
    except ValueError as e:
        print e
        continue
    if current_key != key:
        if current_key == 0:
            idf = 1/log(1 + docs_with_word)
        tf_idf_cur_key = (searched_word_count / word_count)*idf
        current_key = key
        if current_key !=0:
            # if we were computing tf idf for all articles, we would output this
            # print "%s\t%g" % (current_key, tf_idf_cur_key)
            # but the assignment is only for article 12, and we only output the TF IDF
            print "%g" % (tf_idf_cur_key)
    
    if key == 0 and searched_word_count !=0:
        docs_with_word = docs_with_word+1

Overwriting reducer.py


Running it !
We distribute the (small) stop words file

In [12]:
%%bash

OUT_DIR="coursera_mr_task"$(date +"%s%6N")
NUM_REDUCERS=2
NUM_MAPPERS=4

# Stub code for your job

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="TF IDF" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapred.job.maps=${NUM_MAPPERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
# output result to console
hdfs dfs -cat ${OUT_DIR}/part-*

# cleanup result dir
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null



18/11/07 14:27:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/11/07 14:27:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/11/07 14:27:39 INFO mapred.FileInputFormat: Total input files to process : 1
18/11/07 14:27:40 INFO mapreduce.JobSubmitter: number of splits:2
18/11/07 14:27:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1541583469869_0011
18/11/07 14:27:40 INFO impl.YarnClientImpl: Submitted application application_1541583469869_0011
18/11/07 14:27:40 INFO mapreduce.Job: The url to track the job: http://b518dc41edb6:8088/proxy/application_1541583469869_0011/
18/11/07 14:27:40 INFO mapreduce.Job: Running job: job_1541583469869_0011
18/11/07 14:27:45 INFO mapreduce.Job: Job job_1541583469869_0011 running in uber mode : false
18/11/07 14:27:45 INFO mapreduce.Job:  map 0% reduce 0%
18/11/07 14:28:01 INFO mapreduce.Job:  map 7% reduce 0%
18/11/07 14:28:07 INFO mapreduce.Job:  map 10% reduce 0%
18/11/07 14:28:13 INFO m

In [11]:
%%bash

hdfs dfs -ls coursera_mr_task1541600536065098